<a href="https://colab.research.google.com/github/kavyajeetbora/CityHealthMonitor/blob/master/development/built-up-index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install and import external packages

In [1]:
!pip install -q rioxarray
!pip install -q h3
!pip install -q pydeck

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 14.1 MB/s eta 0:00:00


In [2]:
import geemap
import ee
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import pandas as pd
import geopandas as gpd
import rioxarray
from shapely.geometry import Polygon, Point
import h3
from matplotlib import colormaps
import pydeck as pdk
import json
from tqdm import tqdm

ee.Authenticate()
ee.Initialize(project='kavyajeetbora-ee')

## Download the Data

- area of interest
- World Population Estimate dataset from google earth engine

In [3]:
!wget https://github.com/kavyajeetbora/modern_geospatial_stack/raw/master/notebooks/gurgaon.gpkg -O gurgaon.gpkg

--2024-05-14 06:34:05--  https://github.com/kavyajeetbora/modern_geospatial_stack/raw/master/notebooks/gurgaon.gpkg
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kavyajeetbora/modern_geospatial_stack/master/notebooks/gurgaon.gpkg [following]
--2024-05-14 06:34:05--  https://raw.githubusercontent.com/kavyajeetbora/modern_geospatial_stack/master/notebooks/gurgaon.gpkg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 98304 (96K) [application/octet-stream]
Saving to: ‘gurgaon.gpkg’

gurgaon.gpkg        100%[===================>]  96.00K  --.-KB/s    in 0.02s   

2024-05-14 06:34:05 (4.46 MB/s) - ‘g

## Extract area of interest

In [22]:
gurgaon_gdf = gpd.read_file('gurgaon.gpkg', crs='EPSG:4326')
geojson = eval(gurgaon_gdf.to_json())

## Convert the geopackage file to ee.Geometry
featureCollection = ee.FeatureCollection(geojson)
ee_geometry = featureCollection.geometry()

## Get the raster data

In this example we will create a map for urban built-up index

For that we require images from the sentinel satellite

In [37]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

def calculate_indices(image):
    '''
    Calculate the NDVI, NDWI and NDBI for given image and add them to the bands
    '''
    ndvi = image.normalizedDifference(['B8','B4']).rename("NDVI")
    ndwi = image.normalizedDifference(['B8','B11']).rename('NDWI')
    ndbi = image.normalizedDifference(['B11', 'B8']).rename("NDBI")

    return image.addBands(ndvi).addBands(ndwi).addBands(ndbi)

processed = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filterDate('2023-01-01', '2024-01-01')\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
    .map(mask_s2_clouds)\
    .map(calculate_indices)

visualization = {
    'min': -1,
    'max': 1,
    'bands': ['NDBI'],
    'palette': ['#fee0d2','#fc9272','#de2d26']
}

center = (gurgaon_gdf.centroid[0].y, gurgaon_gdf.centroid[0].x)
Map = geemap.Map(center=center, zoom=10)
Map.add_layer(processed.mean().clip(ee_geometry), visualization, 'NDBI')
Map

Map(center=[28.437398083125167, 77.06463665845695], controls=(WidgetControl(options=['position', 'transparent_…

In [6]:
center = (gurgaon_gdf.centroid[0].y, gurgaon_gdf.centroid[0].x)
Map = geemap.Map(center=center, zoom=10)
boundary_viz_params = {
    'color':'red',
    "width":2,
    "lineType":"solid",
    'fillColorOpacity': 0.1
}
Map.addLayer(ee_object=geometry, vis_params=boundary_viz_params, name='Gurgaon Boundary')
viz_params = {
    'min': 0,
    'max': 50,
    'bands': ['population']
}
Map.addLayer(world_pop_image, viz_params, name='Population')
Map

NameError: name 'world_pop_image' is not defined

In [ ]:
geemap.ee_export_image(
    world_pop_image,
    filename='population.tif',
    scale=100,
    file_per_band=False,
    region=geometry,
)

## Polygon to cell using uber-h3

In [18]:
def get_h3_from_boundary(boundary):
    h3_indices = h3.polyfill(boundary, res=9, geo_json_conformant=True)
    df_index = pd.DataFrame(h3_indices, columns=['h3_index'])
    h3_indices = df_index.set_index('h3_index')
    return h3_indices

In [19]:
geojson = eval(gurgaon_gdf.to_json())
geometry = geojson['features'][0]['geometry']
get_h3_from_boundary(geometry)

""
h3_index
893da118c53ffff
893da118eb3ffff
893da11836fffff
893da11aec3ffff
893da103497ffff
...
893da118cb3ffff
893da11832fffff
893da11a373ffff


In [9]:
# Define a layer to display on a map
layer = pdk.Layer(
    "H3HexagonLayer",
    h3_indices.reset_index(),
    pickable=True,
    stroked=True,
    filled=True,
    extruded=False,
    get_hexagon='h3_indices',
    get_fill_color=[143, 32, 45, 100],
    getLineWidth= 20,
    getLineColor = [143, 32, 45, 100]
)

boundary = pdk.Layer(
    "GeoJsonLayer",
    json.loads(gurgaon_gdf.to_json()),
    pickable=True,
    stroked=True,
    filled=False,
    extruded=False,
    get_line_color=[255, 32, 12],
    getLineWidth= 20,

)

# Set the viewport location
aoi = gurgaon_gdf['geometry'].iloc[0]
view_state = pdk.ViewState(latitude=aoi.centroid.y, longitude=aoi.centroid.x, zoom=11, bearing=0, pitch=30)

# Render
r = pdk.Deck(layers=[layer, boundary], initial_view_state=view_state, description='Legend')
r

<IPython.core.display.Javascript object>

{
  "description": "Legend",
  "initialViewState": {
    "bearing": 0,
    "latitude": 28.437398083125167,
    "longitude": 77.06463665845695,
    "pitch": 30,
    "zoom": 11
  },
  "layers": [
    {
      "@@type": "H3HexagonLayer",
      "data": [
        {
          "h3_indices": "893da118c53ffff"
        },
        {
          "h3_indices": "893da118eb3ffff"
        },
        {
          "h3_indices": "893da11836fffff"
        },
        {
          "h3_indices": "893da11aec3ffff"
        },
        {
          "h3_indices": "893da103497ffff"
        },
        {
          "h3_indices": "893da1034abffff"
        },
        {
          "h3_indices": "893da11ad3bffff"
        },
        {
          "h3_indices": "893da11867bffff"
        },
        {
          "h3_indices": "893da11852fffff"
        },
        {
          "h3_indices": "893da118327ffff"
        },
        {
          "h3_indices": "893da118113ffff"
        },
        {
          "h3_indices": "893da11aeafffff"
        },
        {
          "h3_indices": "893da11aaabffff"
        },
        {
          "h3_indices": "893da11ac57ffff"
        },
        {
          "h3_indices": "893da118203ffff"
        },
        {
          "h3_indices": "893da11ab9bffff"
        },
        {
          "h3_indices": "893da118533ffff"
        },
        {
          "h3_indices": "893da118d83ffff"
        },
        {
          "h3_indices": "893da118ca3ffff"
        },
        {
          "h3_indices": "893da118e53ffff"
        },
        {
          "h3_indices": "893da11ac7bffff"
        },
        {
          "h3_indices": "893da1182d3ffff"
        },
        {
          "h3_indices": "893da1036c3ffff"
        },
        {
          "h3_indices": "893da11aecfffff"
        },
        {
          "h3_indices": "893da11854bffff"
        },
        {
          "h3_indices": "893da118343ffff"
        },
        {
          "h3_indices": "893da11a80bffff"
        },
        {
          "h3_indices": "893da118e2fffff"
        },
        {
          "h3_indices": "893da118ed7ffff"
        },
        {
          "h3_indices": "893da11ad63ffff"
        },
        {
          "h3_indices": "893da1036d7ffff"
        },
        {
          "h3_indices": "893da11add3ffff"
        },
        {
          "h3_indices": "893da111aa3ffff"
        },
        {
          "h3_indices": "893da118467ffff"
        },
        {
          "h3_indices": "893da118aa3ffff"
        },
        {
          "h3_indices": "893da1182dbffff"
        },
        {
          "h3_indices": "893da11abd7ffff"
        },
        {
          "h3_indices": "893da118c67ffff"
        },
        {
          "h3_indices": "893da11a8d3ffff"
        },
        {
          "h3_indices": "893da118edbffff"
        },
        {
          "h3_indices": "893da118853ffff"
        },
        {
          "h3_indices": "893da118e47ffff"
        },
        {
          "h3_indices": "893da11a90bffff"
        },
        {
          "h3_indices": "893da1185cfffff"
        },
        {
          "h3_indices": "893da11aa93ffff"
        },
        {
          "h3_indices": "893da11aea7ffff"
        },
        {
          "h3_indices": "893da111e73ffff"
        },
        {
          "h3_indices": "893da11ae93ffff"
        },
        {
          "h3_indices": "893da119dd3ffff"
        },
        {
          "h3_indices": "893da11a97bffff"
        },
        {
          "h3_indices": "893da11883bffff"
        },
        {
          "h3_indices": "893da118517ffff"
        },
        {
          "h3_indices": "893da11ae47ffff"
        },
        {
          "h3_indices": "893da11ada7ffff"
        },
        {
          "h3_indices": "893da11aa0bffff"
        },
        {
          "h3_indices": "893da1195b7ffff"
        },
        {
          "h3_indices": "893da1181afffff"
        },
        {
          "h3_indices": "893da11885bffff"
        },
        {
          "h3_indices": "893da11a8c3ffff"
        },
        {
          "h3_indices":

## Converting the raster to xyz point data


In [ ]:
def raster_to_xyz(raster_file, val_col):
    xds = rioxarray.open_rasterio('population.tif')\
            .sel(band=1)\
            .to_pandas()

    xds = xds.stack().reset_index().rename(columns={"x": 'lng', 'y': 'lat', 0: val_col})
    return xds

def convert_to_gdf(df, aoi):
    df['geometry'] = list(map(Point, zip(df['lng'], df['lat'])))
    gdf = gpd.GeoDataFrame(df, crs='EPSG:4326', geometry='geometry')

    ## Clip the geometry
    gdf = gdf[gdf['geometry'].within(aoi)]
    print(f"Total of {gdf.shape[0]} points are in the dataset")

    return gdf


def aggregate_gdf_by_h3(gdf, APERTURE_SIZE, agg='sum', val_col='population'):
    hex_col = 'hex'+str(APERTURE_SIZE)

    # find hex indices for all the points
    gdf[hex_col] = gdf.apply(lambda x: h3.geo_to_h3(x.lat,x.lng,APERTURE_SIZE),1)

    # calculate temperature average per hex
    hex_data = gdf.groupby(hex_col)[val_col].agg(agg).to_frame(val_col).reset_index()

    return hex_data

1. Convert raster to xyz points

In [ ]:
xds = raster_to_xyz('population.tif', val_col='population', )
xds = xds[xds['population']>0]
print(xds.shape)
print(xds['population'].min(), xds['population'].max())
xds.head()

2. Convert the points to GeoDataFrame

In [ ]:
aoi = gurgaon_gdf['geometry'].iloc[0]
gdf = convert_to_gdf(xds, aoi)
gdf.sample(5)

3. Aggregate all the values by h3 indices

In [ ]:
APERTURE_SIZE=8
pop_hex_data = aggregate_gdf_by_h3(gdf, APERTURE_SIZE=APERTURE_SIZE, agg='sum', val_col='population')
pop_hex_data.sample(3)

After creating the hexagons using the h3 spatial index, the point data are reduced to 3545 from 299484.This saves a significant amount of disk space.

Also compared to geometries which are mostly having coordinates with float numbers, it is more efficient to store a h3 index as it is a string object with fixed number of characters (15-16 characters)

To view the h3 spatial indices, check out this [Uber h3 viewer](https://wolf-h3-viewer.glitch.me/)

## Visualizing the results in pydeck

for visualizing the surface temperature data, we will first format the data


- Normalizing the temperature values
- then color coding the temperature values using a colormap
- splitting the colormap values to make it compatible with pydeck `get_fill_color` attribute
- Formatting a tooltip for interactive data visualization

In [ ]:
def color_code_value(value, cmap):
    color = cmap(value)
    scaled_colors = list(map(lambda x: int(x*255), color[:3]))
    scaled_alpha = int(color[3]*100)

    scaled_colors += [scaled_alpha]
    return scaled_colors

def colormap_dataframe(df, value_col, cmap):

    xdf = df.copy()

    ## Scale the temperature values between 0-1; using MinMax Scaler
    xdf[f'norm_{value_col}'] = (xdf[value_col]-xdf[value_col].min())/(xdf[value_col].max()-xdf[value_col].min())
    xdf['color'] = xdf[f'norm_{value_col}'].apply(lambda x: color_code_value(x, cmap))
    xdf[['R', 'G', 'B', 'A']] = pd.DataFrame(xdf['color'].to_list())
    xdf = xdf.drop(['color', f'norm_{value_col}'], axis=1)

    ## formatting the temperature value upto 1 decimal place
    xdf[value_col] = xdf[value_col].astype(int)

    return xdf

def create_h3_hex_layer(df,hex_col):

    # Define a layer to display on a map
    layer = pdk.Layer(
        "H3HexagonLayer",
        df,
        pickable=True,
        stroked=True,
        filled=True,
        extruded=False,
        get_hexagon=hex_col,
        get_fill_color="[R, G, B, A]"
    )

    return layer

In [ ]:
## Choose a colormap
cmap = colormaps['seismic']
cmap

Generating the tooltip:

In [ ]:
tooltip = {
        "html": "<b>Population</b>: {population}",
        "style": {
            "backgroundColor": "#4CAF50",   # Green shade for background
            "color": "#FFFFFF",             # White for text color
            "border": "2px solid #4CAF50",  # Matching border color
            "borderRadius": "5px",          # Rounded corners
            "boxShadow": "2px 2px 10px rgba(0, 0, 0, 0.2)"  # Soft shadow effect
        }
    }

## Finally plotting the results on a 3D map

In [ ]:
## Apply the colormap to the dataframe
value_col='population'

xdf = colormap_dataframe(df=pop_hex_data, value_col=value_col, cmap=cmap)

## Now create a H3HexagonLayer
hex_col = f'hex{APERTURE_SIZE}'
layer = create_h3_hex_layer(xdf, hex_col)

# Set the viewport location
view_state = pdk.ViewState(latitude=aoi.centroid.y, longitude=aoi.centroid.x, zoom=11, bearing=0, pitch=30)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip=tooltip, description='Legend')
r

In [ ]:
import matplotlib as mpl
COLOR = 'blue'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR

fig, ax = plt.subplots(figsize=(6, 1), layout='constrained')
norm = mpl.colors.Normalize(vmin=xdf['population'].min(), vmax=xdf['population'].max())
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
             cax=ax, orientation='horizontal', label='Some Units')
fig.savefig('temp.png', transparent=True)

## WorldPop Global Project Population Data: Estimated Age and Sex Structures of Residential Population per 100x100m Grid Square

In [ ]:
world_pop_age_sex = ee.ImageCollection("WorldPop/GP/100m/pop_age_sex")\
.filter(ee.Filter.eq('country','IND'))\
.filter(ee.Filter.eq('year', 2020))\
.first().clip(geometry)

geemap.ee_export_image(
    world_pop_age_sex,
    filename='pop_age_sex.tif',
    scale=100,
    file_per_band=False,
    region=geometry,
)

In [ ]:
bandNames = world_pop_age_sex.bandNames().getInfo()
len(bandNames)

In [ ]:
# band_as_variable
xr = rioxarray.open_rasterio('pop_age_sex.tif')
final_df = None
for i, band in enumerate(bandNames):
    pas_df = xr.sel(band=i+1).to_pandas()
    pas_df = pas_df.stack().reset_index().rename(columns={"x": 'lng', 'y': 'lat', 0: band})

    ## Join the dataset
    if i == 0:
        final_df = pas_df
    else:
        final_df = final_df.join(pas_df[[band]])

In [ ]:
final_df.sample(4)

In [ ]:
pas_gdf = convert_to_gdf(final_df, aoi)
pas_gdf.sample(3)

In [ ]:
pas_hexdata = None
APERTURE_SIZE=8
for i, band in tqdm(enumerate(bandNames)):
    if i == 0:
        pas_hexdata = aggregate_gdf_by_h3(pas_gdf, APERTURE_SIZE=APERTURE_SIZE, agg='sum', val_col=band).set_index(f'hex{APERTURE_SIZE}')
    else:
        pas_hexdata2 = aggregate_gdf_by_h3(pas_gdf, APERTURE_SIZE=APERTURE_SIZE, agg='sum', val_col=band).set_index(f'hex{APERTURE_SIZE}')
        pas_hexdata = pas_hexdata.join(pas_hexdata2)

## set population equal to zero for no data
pas_hexdata[pas_hexdata<0]=0

## Set all the variables as int
pas_hexdata = pas_hexdata.astype(int)

## Reset the index
pas_hexdata = pas_hexdata.reset_index()

In [ ]:
pas_hexdata.sample(3)

### Plotting a pie chart to show sex ratio

In [ ]:
male_cols = [col for col in pas_hexdata.columns if col.startswith('M')]
female_cols = [col for col in pas_hexdata.columns if col.startswith('F')]

pas_hexdata['sum_male'] = (pas_hexdata[male_cols].sum(axis=1)/pas_hexdata['population']*100).round()
pas_hexdata['sum_female'] = 100 - pas_hexdata['sum_male']

pas_hexdata.sample(5)

In [ ]:
tooltip = {
        "html": '''<b>Population</b>: {population}<br>
        <b>Male</b>: {sum_male} %<br>
        <b>Female</b>: {sum_female} %
        ''',
        "style": {
            "backgroundColor": "#4CAF50",   # Green shade for background
            "color": "#FFFFFF",             # White for text color
            "border": "2px solid #4CAF50",  # Matching border color
            "borderRadius": "5px",          # Rounded corners
            "boxShadow": "2px 2px 10px rgba(0, 0, 0, 0.2)"  # Soft shadow effect
        }
    }

In [ ]:
## Apply the colormap to the dataframe
value_col='population'

xdf = colormap_dataframe(df=pas_hexdata, value_col=value_col, cmap=cmap)

## Now create a H3HexagonLayer
hex_col = f'hex{APERTURE_SIZE}'
layer = create_h3_hex_layer(xdf, hex_col)

# Set the viewport location
view_state = pdk.ViewState(latitude=aoi.centroid.y, longitude=aoi.centroid.x, zoom=11, bearing=0, pitch=30)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip=tooltip, description='Legend')
r

## Reference

1. [Create a choropleth map using h3 and plotly](https://medium.com/@ransaka/how-to-create-a-choropleth-map-using-uber-h3-plotly-python-c65555744c87)

2. [Uber h3 py notebook](https://github.com/uber/h3-py-notebooks/blob/master/notebooks/unified_data_layers.ipynb)

3. [All available matplotlib colormaps for plotting](https://matplotlib.org/stable/users/explain/colors/colormaps.html)

4. [WorldPop Global Project Population Data: Estimated Residential Population per 100x100m Grid Square](https://developers.google.com/earth-engine/datasets/catalog/WorldPop_GP_100m_pop#description)

5. [Adding a legend to pydeck maps](https://github.com/visgl/deck.gl/issues/4850)

## VIIRS Nighttime Data

Resolution = 463.83 meters, radiance values are in Nano Watts/sr/$cm^2$

**Understanding VIIRS DNB Data:**
1. The VIIRS DNB captures radiance values in the visible and near-infrared spectrum during both day and night.
2. These radiance values represent the amount of light detected by the sensor at nighttime.
3. Higher radiance values indicate brighter sources of light (e.g., urban areas, artificial lighting).
4. Lower radiance values correspond to darker regions with less light.

**Business Decision-Making:**
1. Identify potential retail locations based on radiance patterns.
2. Consider areas with moderate to high radiance (indicating activity) and proximity to potential customers.
3. Avoid areas with extremely low radiance (e.g., remote rural regions).

In [ ]:
viirs = ee.ImageCollection("NOAA/VIIRS/DNB/ANNUAL_V22")\
.filter(ee.Filter.date('2022-01-01', '2023-01-01'))\
.filter(ee.Filter.bounds(geometry)).select('maximum').first()

In [ ]:
geemap.ee_export_image(
    viirs,
    filename='viirs_night_data.tif',
    scale=100,
    file_per_band=False,
    region=geometry,
)

In [ ]:
xds = raster_to_xyz('viirs_night_data.tif', val_col='radiance')
xds = xds[xds['radiance']>0]
print(xds.shape)
print(xds['radiance'].min(), xds['radiance'].max())
xds.head()

In [ ]:
aoi = gurgaon_gdf['geometry'].iloc[0]
gdf = convert_to_gdf(xds, aoi)
gdf.sample(5)

In [ ]:
APERTURE_SIZE=8
radiance_data = aggregate_gdf_by_h3(gdf, APERTURE_SIZE=APERTURE_SIZE, agg='sum', val_col='radiance')
radiance_data.sample(3)

In [ ]:
tooltip = {
        "html": "<b>Radiance</b>: {radiance}",
        "style": {
            "backgroundColor": "#4CAF50",   # Green shade for background
            "color": "#FFFFFF",             # White for text color
            "border": "2px solid #4CAF50",  # Matching border color
            "borderRadius": "5px",          # Rounded corners
            "boxShadow": "2px 2px 10px rgba(0, 0, 0, 0.2)"  # Soft shadow effect
        }
    }

In [ ]:
## Apply the colormap to the dataframe
value_col='radiance'
cmap = colormaps['cividis']

xdf = colormap_dataframe(df=radiance_data, value_col=value_col, cmap=cmap)

## Now create a H3HexagonLayer
hex_col = f'hex{APERTURE_SIZE}'
layer = create_h3_hex_layer(xdf, hex_col)

# Set the viewport location
view_state = pdk.ViewState(latitude=aoi.centroid.y, longitude=aoi.centroid.x, zoom=11, bearing=0, pitch=30)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip=tooltip)
r

## Export the data


In [ ]:
pop_rad_data = radiance_data.set_index(hex_col).join(pas_hexdata.set_index(hex_col))
pop_rad_data = pop_rad_data.reset_index()
pop_rad_data.head()

In [ ]:
pop_rad_data.to_parquet('pop_rad_data.parquet')